# Recursion drawbacks and pitfalls

This tutorial covers pitfalls when designing recursive implementations of algorithms and how to avoid them. Then moving from pitfals to drawbacks of recursion because the latter is not an omnipotent tool to solve problems.

## Table of contents

- [Pitfalls](#pitfalls)

- [Drawbacks](#drawbacks)

- [Workshop](#workshop)

  - [Detecting palindromes](#detecting-palindromes)

  - [Linear search](#linear-search)

- [References](#references)

Importing auxiliary utilities.

In [1]:
from contextlib import nullcontext as does_not_raise
from typing import Any, Callable

import ipytest
import pytest

# https://github.com/chmp/ipytest
ipytest.autoconfig()


def validate_integer(number: Any) -> None:
    """Validates if an object is of int() type."""

    if not isinstance(number, int):
        raise TypeError(f"{number} is not int")


def validate_non_negative_integer(number: int) -> None:
    """Validates a number to be a non-negative integer."""

    validate_integer(number)
    if number < 0:
        raise ValueError(f"{number} < 0")


def validate_positive_integer(number: Any) -> None:
    """Validates a number to be a positive integer."""

    validate_integer(number)
    if number < 1:
        raise ValueError(f"{number} < 1")

### Pitfalls

Recalling the example of an unstoppable recursive function.

In [2]:
def foo():
    foo()


foo()

RecursionError: maximum recursion depth exceeded

No base case, no stop condition, the situation of (call) stack overflow and the function crashes.

This is also possible in case of indirect recursion. One sort of it is when a function invokes itself through a series of other functions calls.

In [ ]:
def hoo():
    foo()

def goo():
    hoo()

def foo():
    goo()


foo()

All right, let's add a base case (or base cases) to stop recursive calls. Nevertheless, only adding base cases without ensuring the inevitable (!) movement of recursive steps to any of the base cases can lead to a stack overflow.

In [ ]:
def countdown(number: int):
    if not number:
        return
    countdown(number - 1)


# negative numbers lead to stack overflow
countdown(-1)

Ah, let's assure that only non-negative numbers can be passed into the countdown function, but the following implementation has a flaw regardless it works fine.

In [ ]:
def countdown(number: int):
    if number < 0:
        raise ValueError(f"{number} < 0")
    if not number:
        print("over")
        return
    countdown(number - 1)


try:
    countdown(-1)
except ValueError as val_err:
    print("negative numbers are intercepted.")
    countdown(0)  # ok
    countdown(100)  # ok as well

The flaw is that checking the incoming number of non-negativity is done every call, which is inefficient. To optimise the solution, let's move apart the validation step to do before the main part.

In [3]:
def countdown(number: int):

    def _cntd(number: int):
        if not number:
            print("over")
            return
        _cntd(number - 1)

    if number < 0:
        raise ValueError(f"{number} < 0")
    _cntd(number)


try:
    countdown(-1)
except ValueError as val_err:
    print("negative numbers are intercepted.")
    countdown(0)  # ok
    countdown(100)  # ok as well

negative numbers are intercepted.
over
over


Yes, it works, cool, however there is another flaw reducing the applicability of the solution almost colossally - the limited size of call stack which is not a bottomless pit or cornucopia.

In [4]:
import sys

recursion_limit = sys.getrecursionlimit()
print(f"Recursion limit -> {recursion_limit}")

countdown(recursion_limit + 1)

Recursion limit -> 3000


RecursionError: maximum recursion depth exceeded

Exactly, the recursion limit on maximum depth with upper-bounds a recursive implementation. Regardless the recursion depth limit can be changed (at least in Python), it's better not to do this cause the system (at least a Python interpreter process) may get corrupted.

So, recursive algorithms have limitations and cannot serve as a panacea.

### Drawbacks

Main pitfalls are left behind, but recursion has its price. Let's compare the performance of a recursive and iterative version of computing the factorial of a non-negative integer number.

In [ ]:
def _rec_fact(nbr: int) -> int:
    """Returns the (nbr)!"""

    if nbr in (0, 1):
        return 1
    return nbr * _rec_fact(nbr - 1)


def factorial_recursive(number: int) -> int:
    """Returns the factorial of a non-negative integer."""

    validate_non_negative_integer(number)
    return _rec_fact(number)


def factorial_iterative(number: int) -> int:
    """Returns the factorial of a non-negative integer."""

    validate_non_negative_integer(number)
    # making it similar to the recursive implementation above
    for nbr in range(number - 1, 2, -1):
        number *= nbr

In [ ]:
%%ipytest

@pytest.mark.parametrize(
    ("number", "answer", "expectation"),
    [
        (-1, None, pytest.raises(ValueError)),
        (0, 1, does_not_raise()),
        (1, 1, does_not_raise()),
        (4, 24, does_not_raise()),
        (5.0, 120, pytest.raises(TypeError)),
        # increase if you dare
        (5000, None, pytest.raises(RecursionError)),
    ],
)
def test_factorial(number, answer, expectation):
    with expectation:
        res_rec = factorial_recursive(number)
        res_iter = factorial_iterative(number)
        res_rec == res_iter == answer

The implementations stand the (unit) tests, but now let's check their performance.

In [5]:
%timeit -r 10 factorial_iterative(100)
%timeit -r 10 factorial_recursive(100)

NameError: name 'factorial_iterative' is not defined

The recursive implementation is about twice times slower that its iterative competitor, so, the time performance cup goes to the iterative version of the algorithm.

Another drawback can be manifested in redundant calls, thereby wasting computing resources. A classic example is a naive implementation of a recursive algorithm for calculating the nth Fibonacci number.

In [6]:
def fibonacci_recursive(nth: int) -> int:
    """Returns the nth Fibonacci number."""

    def _fibrec(nbr: int) -> int:
        """Actual recursive implementation."""

        if not nbr:
            return 0
        if nbr < 3:
            return 1
        return _fibrec(nbr - 2) + _fibrec(nbr - 1)

    validate_non_negative_integer(nth)
    return _fibrec(nth)


def fibonacci_iterative(nth: int) -> int:
    """Returns the nth Fibonacci number."""

    validate_non_negative_integer(nth)
    fibcurr, fibnext = 0, 1
    for _ in range(nth):
        fibcurr, fibnext = fibnext, fibcurr + fibnext
    return fibcurr

In [7]:
%%ipytest


@pytest.mark.parametrize(
    ("number", "answer", "expectation"),
    [
        (-1, None, pytest.raises(ValueError)),
        (0, 0, does_not_raise()),
        (1, 1, does_not_raise()),
        (1.5, None, pytest.raises(TypeError)),
        (2, 1, does_not_raise()),
        (3, 2, does_not_raise()),
        (4, 3, does_not_raise()),
        (5, 5, does_not_raise()),
        (6, 8, does_not_raise()),
    ]
)
def test_fibonacci(number, answer, expectation):
    with expectation:
        res_iter = fibonacci_iterative(number)
        res_rec = fibonacci_recursive(number)
        assert res_iter == res_rec == answer

.........                                                                                    [100%]
9 passed in 0.04s


Time for time performance test.

In [8]:
%timeit -r 10 fibonacci_iterative(10)
%timeit -r 10 fibonacci_recursive(10)

1.24 µs ± 238 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)
8.05 µs ± 31.7 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)


Fantastic, the recusive implementation is as slow as a lazy snail. Welcome to longer computations when increasing the input number. But why so? The following picture may be more illustrative than words:

![Fibonacci calls tree](./fibonacci_calls_tree.png)

There are redundant calls computing the same values, overhead costs can become unreasonably high. Can you count and group the same calls?

There is an optimisation technique called [memoisation](https://www.geeksforgeeks.org/what-is-memoization-a-complete-tutorial/) (not "memorisation", no typo at all). Memoisation allows to cache (store) an input value and the output for it. For this purpose you can declare a dictionary and put there inputs-outputs (key, value) pairs.

In [9]:
MEMO: dict[int, int] = {}


# This is a decorating function.
# It receives a function as an input
# and returns another function.
# This is Decorator pattern in use.
def cache(func: Callable) -> Callable:
    """Caching decorator."""

    def _wrapper(number: int) -> int:
        if number in MEMO:
            return MEMO[number]
        res = func(number)
        MEMO[number] = res  # caching
        return res
    
    return _wrapper


@cache
def fibonacci_recursive_2(nth: int) -> int:
    """Returns the nth Fibonacci number."""

    def _fibrec(nbr: int) -> int:
        """Actual recursive implementation."""

        if not nbr:
            return 0
        if nbr < 3:
            return 1
        return _fibrec(nbr - 2) + _fibrec(nbr - 1)

    validate_non_negative_integer(nth)
    return _fibrec(nth)

The Red Letter Day...

In [10]:
%timeit -r 10 fibonacci_iterative(10)
%timeit -r 10 fibonacci_recursive_2(10)

1.08 µs ± 13.2 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)
630 ns ± 5.17 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)


Wow, the memoised recursive version became faster than the iterative version. Can these results be trusted? Let's memoise the iterative version and make sure that the order of times is aligned to the expected values.

In [11]:
@cache
def fibonacci_iterative_2(nth: int) -> int:
    """Returns the nth Fibonacci number."""

    validate_non_negative_integer(nth)
    fibcurr, fibnext = 0, 1
    for _ in range(nth):
        fibcurr, fibnext = fibnext, fibcurr + fibnext
    return fibcurr


%timeit -r 10 fibonacci_iterative_2(10)
%timeit -r 10 fibonacci_recursive_2(10)

632 ns ± 15.1 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)
631 ns ± 8.16 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)


Now the results are of the same order, which means that outputs are retrieved from the cache, not computed every time the function is invoked.

Can you answer whether it is reasonable to memoise functions all the times? Maybe only recursive functions? What is the price of memoisation?

### Workshop

Although pitfalls and drawbacks of recursion are present, they should not turn you away from recursion - not everything is so clear for final conclusions, so now we keep learning.

0. a brief introduction into computational complexity

1. detecting palindromes

2. linear search

3. raising the number a to the power of b for logarithmic time

Here an attempt will be made to touch on the rudiments of calculating the computational complexity of algorithms, but rather on the fingers without mathematically rigorous justification.

#### BI2CA

Or "a Brief Introduction into Computational Complexity Analysis" to be long enough for a title and an anchor link name.

If are familiar with such words like [computational complexity](https://math.berkeley.edu/~buehler/Computational%20Complexity.pdf), [asymptotic analysis](https://www.educative.io/blog/asymptotic-runtime-complexity-algorithms), [Big-Oh notation](https://stackoverflow.com/questions/487258/what-is-a-plain-english-explanation-of-big-o-notation) and so on, you can confidently move on. If not, you can also do the same, but then study this topic at your leisure, and you don’t have to delve into mathematical analysis (it won’t be here), but it’s very worth getting the basic idea. This topic is often discussed as it relates to measuring the performance of algorithms - how to evaluate their effectiveness depending on the software and hardware.

This section is to be a short introduction into a [Big-Oh](https://www.freecodecamp.org/news/big-o-notation-why-it-matters-and-why-it-doesnt-1674cfa8a23c/) notation or how to evaluate worst-case complexity for an algorithm when the latter must work to its fullest.

Suppose there is an algorithm that we can write as a function `f(n)` where `n` is an input for the algorithm. We would like to understand how much resources - mostly time and computer memory space - it will consume. Let's get started a simple case - constant time and constant space functions.

In [17]:
def constant_time(seq):
    if seq:  # step 1
        return seq[0] == seq[-1]  # 2
    return False  # 3 (may not be reached)


def constant_space(seq):
    return [1, 2, 3]  # step 1


# just a smoke test
for seq in ([], list(range(10))):
    constant_time(seq)
    constant_space(seq)

Why functions? Because they are roughly "named block codes that can take input data and produce output". But this is almost the same thing (roughly writing) as a function in mathematics: something takes input data and produces output data.

The function/algorithm `constant_time` takes a sequence (as intended so) and no matter what size of an input sequence is, either empty or with a n-llion items, it will always take either three steps/instructions/statements (in the first case) or two (in another one). So the algorithm does not care about the size of the input data, its time-complexity is constant and (memory) space complexity as well since no extra memory allocations occur.

The same goes for the function/algorithm `constant_space`: regardless of the input data, it returns a list of three numbers `[1, 2, 3]`. This takes up memory, but the size of the output data is not determined by the input data, and therefore the algorithm/function is considered constant in space complexity. It has the constant time complexity as well - only one return statement is executed.

Such functions are denoted as `O(1)` where `1` is a symbol of constancy. This is a convention to denote functions with constant complexities in constant this way, and not like `O(C)` for not (probably) putting the meaning of variability into the symbol `C` and not measuring functions of this kind by the number of steps or something like that - in the asymptotic sense (for now we use "asynptotic" word without delving into the meaning) they are all about the same thing.

Now let's look at linear functions or linear order complexity. This means that the execution of the algorithm already depends on the size of the input data, but this dependence is linear, in other words, `n` is included as is to the power of 1 and only 1. A prime example is the linear function `f(n) = k * n + b`, where `k` and `b` are constants. This function varies when `n` variable changes, the constant values do not impact . The constants can be large numbers, but they only give a constant correction and still the function depends only on the input variable `n` and is asymptotically (when the `n` tends to infinity) determined only by this input variable.

Such functions are denoted as `O(n)` and here are examples of algorithms of this order of complexity:

In [18]:
def linear_time(seq):
    if seq:
        previous = seq[0]
        for idx in range(1, len(seq)):
            current = seq[idx]
            if current < previous:
                return False
            previous = current
    return True


def linear_space(seq):
    new_seq = []
    for item in seq:
        new_seq.append(item)
    return new_seq


for seq in ([], [1, 2, 3], [4, 4, 5, 5], [6, 0]):
    print(f"seq = {seq}")
    print(f"is_sorted = {linear_time(seq)}, copy = {linear_space(seq)}")

seq = []
is_sorted = True, copy = []
seq = [1, 2, 3]
is_sorted = True, copy = [1, 2, 3]
seq = [4, 4, 5, 5]
is_sorted = True, copy = [4, 4, 5, 5]
seq = [6, 0]
is_sorted = False, copy = [6, 0]


Notable features:

1. for these functions there is a best-case scenario: when the input sequence is empty, no work needs to be done and the answer is given as is and with `O(1)` complexity;
2. in the worst-case branch, all elements of the sequence have to be traversed with no escape from it;

And now the essence: the complexity is determined with the size of the input. An example on the notorious linear function `f(n) = k * n + b`. If `n + m`, then `f(n + m) = k * (n + m) + b = (k * n + b) + m * k = f(n) + C(m)`. The in- or decrement of the function complexity is not constant anymore: it is detemined with `m` term with the coefficient of linearity `k`. So the `C(m)` term also exhibits the linear nature and this is the cost of changing the input in size. In the previous examples, this means the following: **adding `m` element(s) to the original sequence of size `n` will entail `m` iteration(s) each resulting in `k` complexity cost. Constant overhead is not of interest, it is already built into the original function `f(n)`.

#### Detecting palindromes

A [palindrome](https://www.merriam-webster.com/dictionary/palindrome) is such a sequence that, being reversed, is equal to original one. For example, "abcba" is a palyndrome because its reverse is "abcba" again.

1. base cases - an empty sequence or a sequence with one item are already palindromic. There is no less sequence that the empty one, so no need for extra validation;

2. he original sequence should be reduced successfully for each recursive step to bring the recursion to base cases eventually

In [12]:
"""Detects if a sequence is palindromic."""


from typing import Sequence


def is_palindrome_recursive(seq: Sequence) -> bool:
    """Returns True if the sequence is a palindrome.

    Parameters
    ----------
    seq : Sequence

    Returns
    -------
    bool
    """

    if len(seq) < 2:
        return True
    if seq[0] != seq[-1]:
        return False
    return is_palindrome_recursive(seq[1:-1])


def is_palindrome_iterative(seq: Sequence) -> bool:
    """Returns True if the sequence is a palindrome.

    Parameters
    ----------
    seq : Sequence

    Returns
    -------
    bool
    """

    length: int = len(seq)
    if length < 2:
        return True
    for index in range(length // 2):
        if seq[index] != seq[-(index + 1)]:
            return False
    return True


In [13]:
%%ipytest


@pytest.mark.parametrize(
    ("seq", "answer"),
    [
        ([], True),
        ({1}, True),
        ((1, 2), False),
        ((3, 3), True),
        ("abc", False),
        ("aBa", True),
    ]
)
def test_palindrome(seq, answer):
    res_iter = is_palindrome_iterative(seq)
    res_rec = is_palindrome_recursive(seq)
    assert res_iter == res_rec == answer

......                                                                                       [100%]
6 passed in 0.03s


There is an attempt to optimise the algorithm by comparing edge elements, so less steps to take. Nevertheless, the recursive implementation is not optimal enough for each recursive step does copying sequence (slice operator) increasing the overhead. Can you optimise this and get rid of copying?

#### Linear search

A classic warming-up problem on search algorithms. The task is to check wheter a passed element is in a given sequence. If there is a match, the index of the first occurence is returned. Otherwise...I decided to return None, but in case of non-Python-like languages you can return only a number and this can be -1. Returning -1 in Python is tricky because it is a valid value and means "the last element" or "the first element from the end" from a reversed indexation point.

In [14]:
"""The implementations of the linear search algorithm."""


from typing import Any, Sequence


def linsearch_iterative(seq: Sequence[Any], item: Any) -> int | None:
    """Returns the index of the item if it is in the sequence.

    The index for the first occurence if returned.
    If the item is not in the sequence, None is returned.

    Parameters
    ----------
    seq : Sequence[Any]
        where to search
    item : Any
        what to search

    Returns
    -------
    int | None
        the index of the item if found else None.
    """

    for idx, elem in enumerate(seq):
        if elem == item:
            return idx
    return None


def linsearch_recursive(seq: Sequence[Any], item: Any) -> int | None:
    """Returns the index of the item if it is in the sequence.

    The index for the first occurence if returned.
    If the item is not in the sequence, None is returned.

    Parameters
    ----------
    seq : Sequence[Any]
        where to search
    item : Any
        what to search

    Returns
    -------
    int | None
        the index of the item if found else None.
    """

    def _linsearch(start: int, length: int) -> int | None:
        # De Morgan's laws, please check them out on your own
        if not (length and start < length):
            return None
        if seq[start] == item:
            return start
        return _linsearch(start + 1, length)

    return _linsearch(0, len(seq))

In [15]:
%%ipytest


@pytest.mark.parametrize(
    ("sequence", "element", "answer"),
    [
        ([], 1, None),
        ([1], 2, None),
        ([1, 1], 1, 0),
        ([1, 2, 3], 3, 2),
        ((1, 2, 2, 3, 3, 3), 3, 3),
    ],
)
def test_linear_search(sequence, element, answer):
    res_iter = linsearch_iterative(sequence, element)
    res_rec = linsearch_recursive(sequence, element)
    assert res_iter == answer == res_rec

.....                                                                                        [100%]
5 passed in 0.02s


#### fjgklfjgklf


### References

- [Recursion Explained](https://tozturk.hashnode.dev/recursion-explained-breaking-down-the-core-concepts-benefits-and-drawbacks-of-using-recursive-functions)

- [IBM -> Mastering recursive programming](https://developer.ibm.com/articles/l-recurs/)

- [JakeVDP: Timing and Profiling](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html)

- [PyNash: Timing and Profiling](https://pynash.org/2013/03/06/timing-and-profiling/)

- [Free Code Camp -> Memoisation](https://www.freecodecamp.org/news/memoization-in-javascript-and-react/)

- [Free Code Camp -> What is Big O Notation Explained: Space and Time Complexity](https://www.freecodecamp.org/news/big-o-notation-why-it-matters-and-why-it-doesnt-1674cfa8a23c/)